In [6]:
from __future__ import print_function

from keras.models import Model, load_model
from keras.layers import Input
import numpy as np

batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = 'fra-eng/fra.txt'

In [7]:
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text = line.split(' ')
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = ' ' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [33]:
with open('important_data.txt') as json_file:  
    data = json.load(json_file)
    input_token_index=data['input_index']
    target_token_index=data['target_index']
    print(input_token_index)
    for p in data['input_output']: 
        num_encoder_tokens = p['num_encoder_tokens']
        num_decoder_tokens = p['num_decoder_tokens']
        max_encoder_seq_length = p['max_encoder_seq_length']
        max_decoder_seq_length = p['max_decoder_seq_length']

{'\x9d': 0, '¬': 1, 'ঁ': 2, 'ং': 3, 'ঃ': 4, 'অ': 5, 'আ': 6, 'ই': 7, 'ঈ': 8, 'উ': 9, 'ঊ': 10, 'ঋ': 11, 'এ': 12, 'ঐ': 13, 'ও': 14, 'ঔ': 15, 'ক': 16, 'খ': 17, 'গ': 18, 'ঘ': 19, 'ঙ': 20, 'চ': 21, 'ছ': 22, 'জ': 23, 'ঝ': 24, 'ঞ': 25, 'ট': 26, 'ঠ': 27, 'ড': 28, 'ঢ': 29, 'ণ': 30, 'ত': 31, 'থ': 32, 'দ': 33, 'ধ': 34, 'ন': 35, 'প': 36, 'ফ': 37, 'ব': 38, 'ভ': 39, 'ম': 40, 'য': 41, 'র': 42, 'ল': 43, 'শ': 44, 'ষ': 45, 'স': 46, 'হ': 47, 'া': 48, 'ি': 49, 'ী': 50, 'ু': 51, 'ূ': 52, 'ৃ': 53, 'ে': 54, 'ৈ': 55, 'ো': 56, 'ৌ': 57, '্': 58, 'ৎ': 59, 'ৗ': 60, 'ড়': 61, 'ঢ়': 62, 'য়': 63, '\u200c': 64, '\u200d': 65, '\u200f': 66, '\ufeff': 67}


In [34]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
#num_encoder_tokens = len(input_characters)
#num_decoder_tokens = len(target_characters)
#max_encoder_seq_length = max([len(txt) for txt in input_texts])
#max_decoder_seq_length = max([len(txt) for txt in target_texts])

#print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

#input_token_index = dict(
#    [(char, i) for i, char in enumerate(input_characters)])
#target_token_index = dict(
  #  [(char, i) for i, char in enumerate(target_characters)])

print(target_token_index)

#encoder_input_data = np.zeros(
   # (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
  #  dtype='float32')

#for i, input_text in enumerate(input_texts):
 #   for t, char in enumerate(input_text):
  #      encoder_input_data[i, t, input_token_index[char]] = 1.

Number of unique input tokens: 68
Number of unique output tokens: 69
Max sequence length for inputs: 22
Max sequence length for outputs: 22
{'\n': 0, ' ': 1, '\x9d': 2, '¬': 3, 'ঁ': 4, 'ং': 5, 'ঃ': 6, 'অ': 7, 'আ': 8, 'ই': 9, 'ঈ': 10, 'উ': 11, 'ঊ': 12, 'ঋ': 13, 'এ': 14, 'ঐ': 15, 'ও': 16, 'ঔ': 17, 'ক': 18, 'খ': 19, 'গ': 20, 'ঘ': 21, 'ঙ': 22, 'চ': 23, 'ছ': 24, 'জ': 25, 'ঝ': 26, 'ঞ': 27, 'ট': 28, 'ঠ': 29, 'ড': 30, 'ঢ': 31, 'ণ': 32, 'ত': 33, 'থ': 34, 'দ': 35, 'ধ': 36, 'ন': 37, 'প': 38, 'ফ': 39, 'ব': 40, 'ভ': 41, 'ম': 42, 'য': 43, 'র': 44, 'ল': 45, 'শ': 46, 'ষ': 47, 'স': 48, 'হ': 49, 'া': 50, 'ি': 51, 'ী': 52, 'ু': 53, 'ূ': 54, 'ৃ': 55, 'ে': 56, 'ৈ': 57, 'ো': 58, 'ৌ': 59, '্': 60, 'ৎ': 61, 'ৗ': 62, 'ড়': 63, 'ঢ়': 64, 'য়': 65, '\u200c': 66, '\u200d': 67, '\u200f': 68}


In [12]:
import json

data = {}  
data['people'] = []  
data['people'].append({  
    'name': 'Scott',
    'website': 'stackabuse.com',
    'from': 'Nebraska'
})
data['people'].append({  
    'name': 'Larry',
    'website': 'google.com',
    'from': 'Michigan'
})

with open('important_data.txt', 'w') as outfile:  
    json.dump(data, outfile)

In [13]:
with open('important_data.txt') as json_file:  
    data = json.load(json_file)
    for p in data['people']:
        print('Name: ' + p['name'])
        print('Website: ' + p['website'])
        print('From: ' + p['from'])
        print('')

Name: Scott
Website: stackabuse.com
From: Nebraska

Name: Larry
Website: google.com
From: Michigan



In [35]:
# Restore the model and construct the encoder and decoder.
model = load_model('s2s.h5')

encoder_inputs = model.input[0]   # input_1
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output   # lstm_1
encoder_states = [state_h_enc, state_c_enc]
encoder_model = Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]   # input_2
decoder_state_input_h = Input(shape=(latent_dim,), name='input_3')
decoder_state_input_c = Input(shape=(latent_dim,), name='input_5')
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())


# Decodes an input sequence.  Future work should support beam search.
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index[' ']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence


def get_input_data(test_txt):
    #txt = input()
    txt=test_txt
    
    encoder_input_data = np.zeros(
    (len(txt), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
    
    txt = txt.replace(' ', '')
    
    for t, char in enumerate(txt):
        encoder_input_data[0, t, input_token_index[char]] = 1.
        
    return encoder_input_data


word_list=[]
data_path='newspaper(khela).txt'
with open(data_path, 'r', encoding='utf-8') as f:
    #lines = f.read().split(' ')
    for word in f.read().split():
        word_list.append(word)
    for word in word_list:
        input_seq = get_input_data(word)
        decoded_sentence = decode_sequence(input_seq)
        print('Input sentence:',   word)
        print('Decoded sentence:', decoded_sentence)
        
print(len(word_list))        
print(word_list[0])

#for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
 #   input_seq = get_input_data(test_input_texts[i])
 #   input_seq = encoder_input_data[seq_index: seq_index + 1]
  #  decoded_sentence = decode_sequence(input_seq)
   # print('-')
   # print('Input sentence:', input_texts[seq_index])
  #  print('Decoded sentence:', decoded_sentence)

Input sentence: প্রেমাদাসার
Decoded sentence: প্রান্র

Input sentence: মাঠ
Decoded sentence: মালা

Input sentence: থেকে
Decoded sentence: এলা

Input sentence: শুরু
Decoded sentence: বার

Input sentence: করে
Decoded sentence: সে

Input sentence: সামাজিক
Decoded sentence: মালান

Input sentence: যোগাযোগমাধ্যম
Decoded sentence: প্রান

Input sentence: সবখানেই
Decoded sentence: সাল

Input sentence: চলছে
Decoded sentence: এলা

Input sentence: নাগিন
Decoded sentence: মালা

Input sentence: নাচ
Decoded sentence: সাল

Input sentence: কাল
Decoded sentence: সাল

Input sentence: রাতে
Decoded sentence: বার

Input sentence: শ্রীলঙ্কার
Decoded sentence: প্রান্র

Input sentence: বিপক্ষে
Decoded sentence: প্রান্র

Input sentence: অবিস্মরণীয়
Decoded sentence: প্রান

Input sentence: সেই
Decoded sentence: সে

Input sentence: জয়ের
Decoded sentence: মাল

Input sentence: পর
Decoded sentence: বার

Input sentence: মাঠেই
Decoded sentence: মালা

Input sentence: নাগিন
Decoded sentence: মালা

Input sentence: নাচ
Dec

Input sentence: নাজমুলের
Decoded sentence: মালা

Input sentence: এই
Decoded sentence: এএএএএ

Input sentence: নাগিন
Decoded sentence: মালা

Input sentence: নাচ
Decoded sentence: সাল

Input sentence: ত্রিদেশীয়
Decoded sentence: প্রান

Input sentence: সিরিজ
Decoded sentence: সালা

Input sentence: দিয়ে
Decoded sentence: সাল

Input sentence: জায়গা
Decoded sentence: মালা

Input sentence: করে
Decoded sentence: সে

Input sentence: নিয়েছে
Decoded sentence: সাল

Input sentence: আন্তর্জাতিক
Decoded sentence: সার্যান

Input sentence: মঞ্চে
Decoded sentence: বার্রা

Input sentence: বৈশ্বিক
Decoded sentence: প্রান

Input sentence: সংবাদমাধ্যমও
Decoded sentence: মান্রা

Input sentence: এই
Decoded sentence: এএএএএ

Input sentence: নাচকে
Decoded sentence: সালা

Input sentence: লুফে
Decoded sentence: মাল

Input sentence: নিয়েছে
Decoded sentence: সাল

Input sentence: তবে
Decoded sentence: বি

Input sentence: শ্রীলঙ্কা
Decoded sentence: প্রান্র

Input sentence: দলের
Decoded sentence: সাল

Input sentence: স

In [ ]:
# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
print(epochs)
# Run training
#model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
history=model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
# Save model
model.save('test_model.h5')

In [15]:
# Next: inference mode (sampling).
# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
from keras.models import model_from_json

data_path = 'fra-eng/fra.txt'
print("kire")

# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
# load weights into new model
model.load_weights("test_model.h5")
print("Loaded model from disk")
 
# evaluate loaded model on test data
#model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

#model = load_model('s2s.h5')

input_characters=66 #here Number of unique input tokens(characters): 66
num_encoder_tokens = 66
num_decoder_tokens=67
encoder_inputs = Input(shape=(None, num_encoder_tokens))

encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]
encoder_model = Model(encoder_inputs, encoder_states)

decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index[' ']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence




def get_input_data(test_txt):
    #txt = input()
    txt=test_txt
    
    encoder_input_data = np.zeros(
    (len(txt), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
    
    txt = txt.replace(' ', '')
    
    for t, char in enumerate(txt):
        encoder_input_data[0, t, input_token_index[char]] = 1.
        
    return encoder_input_data
    
    
    
    
# Take one sequence (part of the training set)
# for trying out decoding.
#input_seq = encoder_input_data[seq_index: seq_index + 1]

with open(data_path, 'r', encoding='utf-8') as f:
    test_lines = f.read().split('\n')

test_input_texts=[]
test_target_texts=[]
for line in test_lines[3000: 3100]:
    #print(line)
    test_input_text, test_target_text = line.split(' ')
    #print(test_input_text)
    test_input_texts.append(test_input_text)
    test_target_texts.append(test_target_text+'\n')
    
#print(test_input_texts)   

total_test_input=0
total_accuracy=0

for i in range(50):
    total_test_input=total_test_input+1
    input_seq = get_input_data(test_input_texts[i])
    decoded_sentence = decode_sequence(input_seq)
    if (decoded_sentence==test_target_texts[i]):
        total_accuracy=total_accuracy+1
        #print('hoiche')
        
    print('Input sentence:',   test_input_texts[i])
    print('Decoded sentence:', decoded_sentence)
    #print(decoded_sentence[6])
    print('Target sentence:',  test_target_texts[i])
    #print(test_target_texts[i][6])
  
    
print('total_test_input:', total_test_input)
print('total_accuracy:', total_accuracy)
print('result:', (total_accuracy/total_test_input)*100,'%')

#yhat = model.predict(input_seq, verbose=0)
#print(yhat)

kire


ValueError: Dimension 0 in both shapes must be equal, but are 66 and 63. Shapes are [66,1024] and [63,1024]. for 'Assign_8' (op: 'Assign') with input shapes: [66,1024], [63,1024].

In [21]:
a=2

In [22]:
a=3

In [23]:
print(a)

3
